# Product Recommendation with Feathr

This notebook illustrates the use of Feathr Feature Store to create a model that predict users' rating for different products for a e-commerce website.

### Model Problem Statement
The e-commerce website has collected past user ratings for various products. The website also collected data about user and product, like user age, product category etc. Now we want to predict users' product rating for new product so that we can recommend the new product to users that give a high rating for those products.

### Feature Creation Illustration
In this example, our observation data has compound entity key where a record is uniquely identified by `user_id` and `product_id`. With that, we can think about three types of features:
1. **User features** that are different for different users but are the same for different products. For example, user age is different for different users but it's product-agnostic.
2. **Product features** that are different for different products but are the same for all the users.
3. **User-to-product** features that are different for different users AND different products. For example, a feature to represent if the user has bought this product before or not.

In this example, we will focus on the first two types of features. After we train a model based on those features, we predict the product ratings that users will give for the products.

The feature creation flow is as below:
![Feature Flow](https://github.com/feathr-ai/feathr/blob/main/docs/images/product_recommendation_advanced.jpg?raw=true)

## 2. Config Feathr Client

In [1]:
import glob
import os
import tempfile
from datetime import datetime, timedelta
from math import sqrt

import pandas as pd
from pyspark.sql import DataFrame
# from interpret.provider import InlineProvider
# from interpret import set_visualize_provider

# set_visualize_provider(InlineProvider())

import feathr
from feathr import (
    FeathrClient,
    BOOLEAN, FLOAT, INT32, ValueType,
    Feature, DerivedFeature, FeatureAnchor,
    BackfillTime, MaterializationSettings,
    FeatureQuery, ObservationSettings,
    RedisSink,
    INPUT_CONTEXT, HdfsSource,
    WindowAggTransformation,
    TypedKey,
)
from feathr.datasets.constants import (
    PRODUCT_RECOMMENDATION_USER_OBSERVATION_URL,
    PRODUCT_RECOMMENDATION_USER_PROFILE_URL,
    PRODUCT_RECOMMENDATION_USER_PURCHASE_HISTORY_URL,
    PRODUCT_RECOMMENDATION_PRODUCT_DETAIL_URL,
)
from feathr.datasets.utils import maybe_download
from feathr.utils.config import generate_config
from feathr.utils.job_utils import get_result_df

print(f"Feathr version: {feathr.__version__}")

Feathr version: 1.0.0


In [2]:
os.environ['SPARK_LOCAL_IP'] = "127.0.0.1"
os.environ['REDIS_PASSWORD'] = "foobared"
PROJECT_NAME = "product_recommendation_from_s3"

### Initialize Feathr Client

In [3]:
from pathlib import Path
feathr_workspace_folder = Path(f"./{PROJECT_NAME}_feathr_config.yaml")
client = FeathrClient(str(feathr_workspace_folder))

2024-09-06 23:11:54.403 | INFO     | feathr.utils._env_config_reader:get:62 - Config secrets__azure_key_vault__name is not found in the environment variable, configuration file, or the remote key value store. Returning the default value: None.
2024-09-06 23:11:54.409 | INFO     | feathr.utils._env_config_reader:get:62 - Config offline_store__s3__s3_enabled is not found in the environment variable, configuration file, or the remote key value store. Returning the default value: None.
2024-09-06 23:11:54.410 | INFO     | feathr.utils._env_config_reader:get:62 - Config offline_store__adls__adls_enabled is not found in the environment variable, configuration file, or the remote key value store. Returning the default value: None.
2024-09-06 23:11:54.412 | INFO     | feathr.utils._env_config_reader:get:62 - Config offline_store__wasb__wasb_enabled is not found in the environment variable, configuration file, or the remote key value store. Returning the default value: None.
2024-09-06 23:11:54

### What's a feature in Feathr
A feature is an individual measurable property or characteristic of a phenomenon which is sometimes time-sensitive.

In Feathr, a feature is defined by the following characteristics:
* The typed key (a.k.a. entity id): identifies the subject of feature, e.g. a user id of 123, a product id of SKU234456.
* The feature name: the unique identifier of the feature, e.g. user_age, total_spending_in_30_days.
* The feature value: the actual value of that aspect at a particular time, e.g. the feature value of the person's age is 30 at year 2022.
* The timestamp: this indicates when the event happened. For example, the user purchased certain product on a certain timestamp. This is usually used for point-in-time join.

You can feel that this is defined from a feature consumer (a person who wants to use a feature) perspective. It only tells us what a feature is like. In later sections, you can see how a feature consumer can access the features in a very simple way.

To define how to produce the feature, we need to specify:
* Feature source: what source data that this feature is based on
* Transformation: what transformation is used to transform the source data into feature. Transformation can be optional when you just want to take a column out from the source data.

(For more details on feature definition, please refer to the [Feathr Feature Definition Guide](https://feathr-ai.github.io/feathr/concepts/feature-definition.html).)

Note: in some cases, such as features defined on top of request data, may have no entity key or timestamp.
It is merely a function/transformation executing against request data at runtime.
For example, the day of week of the request, which is calculated by converting the request UNIX timestamp.
(We won't cover this in the tutorial.)

### Define Sources Section with UDFs

A feature is called an anchored feature when the feature is directly extracted from the source data, rather than computed on top of other features. The latter case is called derived feature.

A [feature source](https://feathr.readthedocs.io/en/latest/#feathr.Source) is needed for anchored features that describes the raw data in which the feature values are computed from. See the python documentation to get the details on each input column.

See [the python API documentation](https://feathr.readthedocs.io/en/latest/#feathr.HdfsSource) to get the details of each input fields.

### Define window aggregation features

[Window aggregation](https://en.wikipedia.org/wiki/Window_function_%28SQL%29) helps us to create more powerful features by compressing large amount of information. For example, we can compute *average purchase amount over the last 90 days* from the purchase history to capture user's recent consumption trend.

To create window aggregation features, we define `WindowAggTransformation` with following arguments:
1. `agg_expr`: the field/column you want to aggregate. It can be an ANSI SQL expression, e.g. `cast_float(purchase_amount)` to cast `str` type values to `float`.
2. `agg_func`: the aggregation function, e.g. `AVG`. See below table for the full list of supported functions.
3. `window`: the aggregation window size, e.g. `90d` to aggregate over the 90 days.

| Aggregation Type | Input Type | Description |
| --- | --- | --- |
| `SUM`, `COUNT`, `MAX`, `MIN`, `AVG` | Numeric | Applies the the numerical operation on the numeric inputs. |
| `MAX_POOLING`, `MIN_POOLING`, `AVG_POOLING`	| Numeric Vector | Applies the max/min/avg operation on a per entry basis for a given a collection of numbers. |
| `LATEST` | Any | Returns the latest not-null values from within the defined time window. |

After you have defined features and sources, bring them together to build an anchor:

> Note that if the features comes directly from the observation data, the `source` argument should be `INPUT_CONTEXT` to indicate the source of the anchor is the observation data.

## Get features from Register Server

In [4]:
client.list_registered_features(project_name=PROJECT_NAME)

[{'name': 'feature_user_age',
  'id': '4ab9154c-e5ec-4897-8744-6e5731e40f96',
  'qualifiedName': 'product_recommendation_from_s3__anchored_features__feature_user_age'},
 {'name': 'feature_user_gift_card_balance',
  'id': '6b41d544-d983-46f6-ad1b-8b04afb75467',
  'qualifiedName': 'product_recommendation_from_s3__anchored_features__feature_user_gift_card_balance'},
 {'name': 'feature_product_quantity',
  'id': 'a00b54ca-cfcb-4e65-b36d-8890cc5221c6',
  'qualifiedName': 'product_recommendation_from_s3__product_anchored_features__feature_product_quantity'},
 {'name': 'feature_user_purchasing_power',
  'id': '3d611a32-883e-4be1-ae58-45dfc4cdbf7e',
  'qualifiedName': 'product_recommendation_from_s3__feature_user_purchasing_power'},
 {'name': 'feature_user_avg_purchase_for_90days',
  'id': 'fff49cb3-2e0c-45ae-bd60-093d868e2b78',
  'qualifiedName': 'product_recommendation_from_s3__aggregationFeatures__feature_user_avg_purchase_for_90days'},
 {'name': 'feature_user_tax_rate',
  'id': '2d344b29-7

In [5]:
feature_dict = client.get_features_from_registry(project_name=PROJECT_NAME, return_keys=True)

## we can list all features

In [6]:
[feat.name for feat in list(feature_dict[0].values())]

['feature_user_avg_purchase_for_90days',
 'feature_user_age',
 'feature_user_tax_rate',
 'feature_user_gift_card_balance',
 'feature_user_has_valid_credit_card',
 'feature_product_quantity',
 'feature_product_price',
 'feature_user_purchasing_power']

## we can list all type_key

In [7]:
[type_key.key_column for type_keys in list(feature_dict[1].values()) for type_key in type_keys]

['user_id',
 'user_id',
 'user_id',
 'user_id',
 'user_id',
 'product_id',
 'product_id',
 'user_id']

Let's use the helper function `get_result_df` to download the result and view it:

## 6. Feature Materialization

While Feathr can compute the feature value from the feature definition on-the-fly at request time, it can also pre-compute
and materialize the feature value to offline and/or online storage.

We can push the generated features to the online store like below:

In [8]:
require_feature_names = ['user_id', 'product_id', 'feature_user_avg_purchase_for_90days',
       'feature_product_price', 'feature_product_quantity',
       'feature_user_gift_card_balance', 'feature_user_has_valid_credit_card',
       'feature_user_tax_rate', 'feature_user_age',
       'feature_user_purchasing_power']

In [9]:
os.environ['S3_ENDPOINT'] = "127.0.0.1:9000"
os.environ['S3_ACCESS_KEY'] = "zDaLfup26Qf2psWSVksV"
os.environ['S3_SECRET_KEY'] = "0uP2ohdTWeLXzV5UAlF8z5JVn17aEbr6OmOa9dkv"

In [10]:
# user key 

# user_feature_names = ['feature_user_avg_purchase_for_90days',
#        'feature_user_gift_card_balance', 'feature_user_has_valid_credit_card',
#        'feature_user_tax_rate', 'feature_user_age',
#        'feature_user_purchasing_power']
feature_dict = client.get_features_from_registry(project_name=PROJECT_NAME, return_keys=True)
key_user_id = feature_dict[1]["feature_user_avg_purchase_for_90days"][0]
user_feature_names = [fea for fea in feature_dict[1] if (feature_dict[1][fea][0].key_column == key_user_id.key_column) ]



In [11]:
# Materialize user features
# Note, you can only materialize features of same entity key into one table.
redisSink = RedisSink(table_name="user_features")
settings = MaterializationSettings(
    name="user_feature_setting",
    sinks=[redisSink],
    feature_names=user_feature_names,
)

client.materialize_features(settings=settings,
                            allow_materialize_non_agg_feature=True,
                            execution_configurations={
                              "spark.hadoop.fs.s3a.endpoint": "http://127.0.0.1:9000",
                              "spark.hadoop.fs.s3a.impl": "org.apache.hadoop.fs.s3a.S3AFileSystem",
                              "spark.hadoop.fs.s3a.path.style.access": "true",
                              'spark.hadoop.fs.s3a.aws.credentials.provider':'org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider',
                              "spark.jars.plus.packages": 
                                                       "io.minio:spark-select_2.11:2.1,org.apache.hadoop:hadoop-aws:3.3.2"
                             
                                                     }
                           )
client.wait_job_to_finish(timeout_sec=300)

2024-09-06 23:11:55.232 | WARNING  | feathr.utils._env_config_reader:get_from_env_or_akv:88 - Config KAFKA_SASL_JAAS_CONFIG is not found in the environment variable or the remote key value store.
2024-09-06 23:11:55.234 | INFO     | feathr.utils._env_config_reader:get:62 - Config monitoring__database__sql__url is not found in the environment variable, configuration file, or the remote key value store. Returning the default value: None.
2024-09-06 23:11:55.235 | INFO     | feathr.utils._env_config_reader:get:62 - Config monitoring__database__sql__user is not found in the environment variable, configuration file, or the remote key value store. Returning the default value: None.
2024-09-06 23:11:55.245 | WARNING  | feathr.utils._env_config_reader:get_from_env_or_akv:88 - Config MONITORING_DATABASE_SQL_PASSWORD is not found in the environment variable or the remote key value store.
2024-09-06 23:11:55.246 | WARNING  | feathr.spark_provider._localspark_submission:submit_feathr_job:78 - Loca

x

https://repository.mulesoft.org/nexus/content/repositories/public/ added as a remote repository with the name: repo-1
https://linkedin.jfrog.io/artifactory/open-source/ added as a remote repository with the name: repo-2
Ivy Default Cache set to: /Users/cuongtranhuu/.ivy2/cache
The jars for the packages stored in: /Users/cuongtranhuu/.ivy2/jars
org.apache.spark#spark-avro_2.12 added as a dependency
org.apache.logging.log4j#log4j-core added as a dependency
com.typesafe#config added as a dependency
org.apache.hadoop#hadoop-mapreduce-client-core added as a dependency
org.apache.hadoop#hadoop-common added as a dependency
org.apache.avro#avro added as a dependency
org.apache.xbean#xbean-asm6-shaded added as a dependency
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.apache.kafka#kafka-clients added as a dependency
com.jasonclawson#jackson-dataformat-hocon added as a dependency
com.redislabs#spark-redis_2.12 added as a dependency
org.apache.xbean#xbean-asm6-shaded added 

>>>>>>>>>>>>x

2024-09-06 23:13:10.402 | INFO     | feathr.spark_provider._localspark_submission:wait_for_completion:198 - Pyspark job Completed


>

2024-09-06 23:13:11.409 | INFO     | feathr.spark_provider._localspark_submission:wait_for_completion:234 - Spark job with pid 1888 finished in: 76 seconds.


>

We can then get the features from the online store (Redis):

In [12]:
client.get_online_features(
    "user_features", "2", user_feature_names
)

[0.0, 44, 0.009999999776482582, 880.0, True, 980.0]

In [13]:
client.multi_get_online_features(
    "user_features", ["1", "2", "34", "45", "78"], user_feature_names
)

{'1': [0.0, 56, 0.009999999776482582, 805.0, True, 905.0],
 '2': [0.0, 44, 0.009999999776482582, 880.0, True, 980.0],
 '34': [0.0, 20, 0.05000000074505806, 465.0, True, 565.0],
 '45': [0.0, 49, 0.019999999552965164, 1229.0, True, 1329.0],
 '78': [0.0, 41, 0.009999999776482582, 1652.0, True, 1752.0]}

In [14]:
# product key

# product_feature_names = [
#        'feature_product_price', 'feature_product_quantity',
# ]
feature_dict = client.get_features_from_registry(project_name=PROJECT_NAME, return_keys=True)
key_product_id = feature_dict[1]["feature_product_quantity"][0]
product_feature_names = [fea for fea in feature_dict[1] if (feature_dict[1][fea][0].key_column == key_product_id.key_column) ]

In [15]:
# Materialize user features
# Note, you can only materialize features of same entity key into one table.
product_redisSink = RedisSink(table_name="product_features1")
product_settings = MaterializationSettings(
    name="product_feature_setting",
    sinks=[product_redisSink],
    feature_names=product_feature_names,
)

client.materialize_features(settings=product_settings,
                            allow_materialize_non_agg_feature=True,
                            execution_configurations={ 
                              "spark.hadoop.fs.s3a.endpoint": "http://127.0.0.1:9000",
                              "spark.hadoop.fs.s3a.impl": "org.apache.hadoop.fs.s3a.S3AFileSystem",
                              "spark.hadoop.fs.s3a.path.style.access": "true",
                              'spark.hadoop.fs.s3a.aws.credentials.provider':'org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider',
                              "spark.jars.plus.packages": 
                                                       "io.minio:spark-select_2.11:2.1,org.apache.hadoop:hadoop-aws:3.3.2"
                             
                                                     }
                           )
client.wait_job_to_finish(timeout_sec=300)

2024-09-06 23:13:11.803 | WARNING  | feathr.utils._env_config_reader:get_from_env_or_akv:88 - Config KAFKA_SASL_JAAS_CONFIG is not found in the environment variable or the remote key value store.
2024-09-06 23:13:11.805 | INFO     | feathr.utils._env_config_reader:get:62 - Config monitoring__database__sql__url is not found in the environment variable, configuration file, or the remote key value store. Returning the default value: None.
2024-09-06 23:13:11.805 | INFO     | feathr.utils._env_config_reader:get:62 - Config monitoring__database__sql__user is not found in the environment variable, configuration file, or the remote key value store. Returning the default value: None.
2024-09-06 23:13:11.808 | WARNING  | feathr.utils._env_config_reader:get_from_env_or_akv:88 - Config MONITORING_DATABASE_SQL_PASSWORD is not found in the environment variable or the remote key value store.
2024-09-06 23:13:11.809 | WARNING  | feathr.spark_provider._localspark_submission:submit_feathr_job:78 - Loca

x

https://repository.mulesoft.org/nexus/content/repositories/public/ added as a remote repository with the name: repo-1
https://linkedin.jfrog.io/artifactory/open-source/ added as a remote repository with the name: repo-2
Ivy Default Cache set to: /Users/cuongtranhuu/.ivy2/cache
The jars for the packages stored in: /Users/cuongtranhuu/.ivy2/jars
org.apache.spark#spark-avro_2.12 added as a dependency
org.apache.logging.log4j#log4j-core added as a dependency
com.typesafe#config added as a dependency
org.apache.hadoop#hadoop-mapreduce-client-core added as a dependency
org.apache.hadoop#hadoop-common added as a dependency
org.apache.avro#avro added as a dependency
org.apache.xbean#xbean-asm6-shaded added as a dependency
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.apache.kafka#kafka-clients added as a dependency
com.jasonclawson#jackson-dataformat-hocon added as a dependency
com.redislabs#spark-redis_2.12 added as a dependency
org.apache.xbean#xbean-asm6-shaded added 

>>>>x

2024-09-06 23:14:18.880 | INFO     | feathr.spark_provider._localspark_submission:wait_for_completion:198 - Pyspark job Completed


>

2024-09-06 23:14:19.884 | INFO     | feathr.spark_provider._localspark_submission:wait_for_completion:234 - Spark job with pid 3095 finished in: 68 seconds.


>

In [16]:
product_feature_names

['feature_product_quantity', 'feature_product_price']

In [17]:
client.get_online_features(
    "product_features1", "1", product_feature_names
)

[42.0, 104.22636413574219]

In [18]:
client.multi_get_online_features(
    "product_features1", [str(i) for i in range(1,100,1)], product_feature_names
)

{'1': [42.0, 104.22636413574219],
 '2': [69.0, 73.68902587890625],
 '3': [21.0, 46.5704460144043],
 '4': [98.0, 52.39152145385742],
 '5': [12.0, 38.17158889770508],
 '6': [14.0, 127.15583801269531],
 '7': [31.0, 32.73922348022461],
 '8': [30.0, 107.25607299804688],
 '9': [2.0, 170.65872192382812],
 '10': [93.0, 137.5769500732422],
 '11': [69.0, 9.159915924072266],
 '12': [21.0, 138.4503631591797],
 '13': [98.0, 103.42092895507812],
 '14': [53.0, 99.29521179199219],
 '15': [6.0, 112.33995819091797],
 '16': [45.0, 31.68714714050293],
 '17': [33.0, 126.61576080322266],
 '18': [73.0, 28.6544246673584],
 '19': [15.0, 154.49440002441406],
 '20': [60.0, 163.63510131835938],
 '21': [26.0, 146.3370361328125],
 '22': [84.0, 189.39381408691406],
 '23': [79.0, 60.23202133178711],
 '24': [8.0, 24.65973663330078],
 '25': [11.0, 2.8535869121551514],
 '26': [28.0, 74.95919799804688],
 '27': [58.0, 124.24835968017578],
 '28': [24.0, 135.34925842285156],
 '29': [70.0, 149.00674438476562],
 '30': [41.0, 

# get features for infer step

In [19]:
infer_user_features = client.multi_get_online_features(
    "user_features", ["1", "2", "34", "45", "78"], user_feature_names
)
infer_user_features_df = pd.DataFrame(infer_user_features).T.reset_index()
infer_user_features_df.columns =["user_id"] + user_feature_names

In [20]:
infer_product_features = client.multi_get_online_features(
    "product_features1", [str(i) for i in range(1,100,1)], product_feature_names
)
infer_product_features_df = pd.DataFrame(infer_product_features).T.reset_index()
infer_product_features_df.columns =["product_id"] + product_feature_names

In [21]:
infer_user_features_df["cross_key"] = 1
infer_product_features_df["cross_key"] = 1

In [22]:
features_df = pd.merge(infer_user_features_df, infer_product_features_df, on ='cross_key').drop(labels="cross_key", axis=1)

In [23]:
order_features = ['user_id', 'product_id', 'feature_user_avg_purchase_for_90days',
       'feature_product_price', 'feature_product_quantity',
       'feature_user_gift_card_balance', 'feature_user_has_valid_credit_card',
       'feature_user_tax_rate', 'feature_user_age',
       'feature_user_purchasing_power']

In [24]:
features_df[order_features]

,user_id,product_id,feature_user_avg_purchase_for_90days,feature_product_price,feature_product_quantity,feature_user_gift_card_balance,feature_user_has_valid_credit_card,feature_user_tax_rate,feature_user_age,feature_user_purchasing_power
0,1,1,0.0,104.226364,42.0,805.0,True,0.01,56,905.0
1,1,2,0.0,73.689026,69.0,805.0,True,0.01,56,905.0
2,1,3,0.0,46.570446,21.0,805.0,True,0.01,56,905.0
3,1,4,0.0,52.391521,98.0,805.0,True,0.01,56,905.0
4,1,5,0.0,38.171589,12.0,805.0,True,0.01,56,905.0
...,...,...,...,...,...,...,...,...,...,...
490,78,95,0.0,135.246643,10.0,1652.0,True,0.01,41,1752.0
491,78,96,0.0,151.344635,49.0,1652.0,True,0.01,41,1752.0
492,78,97,0.0,170.715408,20.0,1652.0,True,0.01,41,1752.0
493,78,98,0.0,7.136273,50.0,1652.0,True,0.01,41,1752.0
